In [2]:
from pathlib import Path
import json
import geopandas as gpd
import pandas as pd
import netCDF4

In [3]:
basePath = Path.cwd().parent.parent.parent / 'inputs'
shpFilePath = basePath / 'global_osm_rails'
osmRailDf = gpd.read_file(shpFilePath / 'global_osm_rails.shp')

In [4]:
import xarray as xr

diffNcPath = basePath / 'spear_tas_diff.nc'


tasDiffNc = netCDF4.Dataset(diffNcPath)
tasDiffdf = xr.open_dataset(diffNcPath).to_dataframe()
# tasDiffdf.index.names =['geo_center_x_zone', 'geo_center_y_zone']
tasDiffdf = tasDiffdf.reset_index()
tasDiffdf = gpd.GeoDataFrame(
  tasDiffdf, geometry=gpd.points_from_xy(tasDiffdf.lon, tasDiffdf.lat), crs="EPSG:4326" 
)

In [ ]:
osmRailDf['geo_center'] = osmRailDf.centroid

In [ ]:
gbRail = osmRailDf.cx[-1.57216793459:1.68153079591,48.959999905:56]
tasDiffdf

In [ ]:
joined = gpd.sjoin_nearest(osmRailDf, tasDiffdf)

In [ ]:
joined.explore(
    legend= True,
    column= 'tas',
    prefer_canvas=True
).save('test.html')